This Notebook will compare the differences in performance between using a UDF and a native pyspark syntax

We will use a **UDF** to create a new column that contains the profit percentage. However, in practice, it is generally better to use native PySpark methods, like the withColumn function, as they offer significantly better performance. This is primarily because they avoid the overhead of serialization and deserialization, which is minimized further with the use of Apache Arrow. But, we'll use a UDF here purely for illustrative purposes.

If we do choose to use a PySpark UDF, we have three main options:

- Row-at-a-Time UDF: Which is inefficient for our case since we have a large amount of transactional data (Fact Table) 

- Pandas UDF: Vectorized and significantly faster, utilizing Apache Arrow to reduce serialization and deserialization, and is way faster than Row-at-a-Time UDF.

- Pandas Function API: Which for our case is an overkill, since we are not doing a complex transformation neither processing a grouped data.

So Pandas UDF will be our choice.

In [0]:
#Load Data
options = {
    "header": "true",
    "inferSchema": "true",
    "delimiter": ";"
} 
dim_product =  spark.read.format("csv").options(**options).load("dbfs:/FileStore/tables_new/dim_product/")
fact_int_sales =  spark.read.format("csv").options(**options).load("dbfs:/FileStore/tables_new/fact_int_sales/")
dim_sales_terr =  spark.read.format("csv").options(**options).load("dbfs:/FileStore/tables_new/dim_sales_terr/")
dim_curreny = spark.read.format("csv").options(**options).load("dbfs:/FileStore/tables_new/dim_curreny/")  

In [0]:
from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import DoubleType 
import pandas as pd

@pandas_udf(DoubleType())
def profit_percent(ProductCost: pd.Series,SalesAmount: pd.Series) -> pd.Series:
    return  ((SalesAmount - ProductCost)/SalesAmount) # Vectorized operation on entire column

In [0]:
# Apply UDF
fact_int_sales= fact_int_sales.withColumn("ProfitPercent", profit_percent(fact_int_sales["TotalProductCost"], fact_int_sales["SalesAmount"]))
fact_int_sales.show()

+----------+------------+----------+-----------+-----------+------------+-----------+-----------------+----------------+--------------------+---------+--------------+-------------------+----------------+-----------+------+-------+----------+----------+----------+------------------+
|ProductKey|OrderDateKey|DueDateKey|ShipDateKey|CustomerKey|PromotionKey|CurrencyKey|SalesTerritoryKey|SalesOrderNumber|SalesOrderLineNumber|UnitPrice|ExtendedAmount|ProductStandardCost|TotalProductCost|SalesAmount|TaxAmt|Freight| OrderDate|   DueDate|  ShipDate|     ProfitPercent|
+----------+------------+----------+-----------+-----------+------------+-----------+-----------------+----------------+--------------------+---------+--------------+-------------------+----------------+-----------+------+-------+----------+----------+----------+------------------+
|       310|    20101229|  20110110|   20110105|      21768|           1|         19|                6|         SO43697|                   1|  3578.27|

In [0]:
# Apply Transformation Without udf
fact_int_sales= fact_int_sales.withColumn("ProfitPercent",  (fact_int_sales["SalesAmount"] - fact_int_sales["TotalProductCost"])/fact_int_sales["SalesAmount"])
fact_int_sales.show()

In [0]:
display(fact_int_sales.limit(10))

ProductKey,OrderDateKey,DueDateKey,ShipDateKey,CustomerKey,PromotionKey,CurrencyKey,SalesTerritoryKey,SalesOrderNumber,SalesOrderLineNumber,UnitPrice,ExtendedAmount,ProductStandardCost,TotalProductCost,SalesAmount,TaxAmt,Freight,OrderDate,DueDate,ShipDate,ProfitPercent
310,20101229,20110110,20110105,21768,1,19,6,SO43697,1,3578.27,3578.27,2171.29,2171.29,3578.27,286.26,894.568,2010-12-29,2011-01-10,2011-01-05,0.3932011838122892
346,20101229,20110110,20110105,28389,1,39,7,SO43698,1,3399.99,3399.99,1912.15,1912.15,3399.99,272.0,849.998,2010-12-29,2011-01-10,2011-01-05,0.4376012870626089
346,20101229,20110110,20110105,25863,1,100,1,SO43699,1,3399.99,3399.99,1912.15,1912.15,3399.99,272.0,849.998,2010-12-29,2011-01-10,2011-01-05,0.4376012870626089
336,20101229,20110110,20110105,14501,1,100,4,SO43700,1,null,null,413.15,413.15,699.1,55.93,174.775,2010-12-29,2011-01-10,2011-01-05,0.4090258904305536
346,20101229,20110110,20110105,11003,1,6,9,SO43701,1,3399.99,3399.99,1912.15,1912.15,3399.99,272.0,849.998,2010-12-29,2011-01-10,2011-01-05,0.4376012870626089
311,20101230,20110111,20110106,27645,1,100,4,SO43702,1,3578.27,3578.27,2171.29,2171.29,3578.27,286.26,894.568,2010-12-30,2011-01-11,2011-01-06,0.3932011838122892
310,20101230,20110111,20110106,16624,1,6,9,SO43703,1,3578.27,3578.27,2171.29,2171.29,3578.27,286.26,894.568,2010-12-30,2011-01-11,2011-01-06,0.3932011838122892
351,20101230,20110111,20110106,11005,1,6,9,SO43704,1,3374.99,3374.99,1898.09,1898.09,3374.99,270.0,843.748,2010-12-30,2011-01-11,2011-01-06,0.4376012965964344
344,20101230,20110111,20110106,11011,1,6,9,SO43705,1,3399.99,3399.99,1912.15,1912.15,3399.99,272.0,849.998,2010-12-30,2011-01-11,2011-01-06,0.4376012870626089
312,20101231,20110112,20110107,27621,1,100,4,SO43706,1,3578.27,3578.27,2171.29,2171.29,3578.27,286.26,894.568,2010-12-31,2011-01-12,2011-01-07,0.3932011838122892
